# Tests for Flatten Layer

To make sure that the output from the GPU is correct, you should test them against what you get with the original CPU implementation. The following code is what you get from the original implementation on the CPU.

In [19]:
import numpy as np

import PyNetwork
import pyopencl as cl
import pyopencl.array as cl_array

In [20]:
input_shape = (10, 9)
output_nodes = 90

platform = cl.get_platforms()
devices = platform[0].get_devices()
context = cl.Context(devices)
queue = cl.CommandQueue(context)
layer = PyNetwork.layers.Flatten()
layer.build(context, queue, input_shape)

# Test Predict

In [21]:
from PyNetwork import utils

In [22]:
n_datapoints = 20
z_in = np.random.rand(n_datapoints, *input_shape).astype(np.float32) - 0.5
z_in_gpu = cl_array.to_device(queue, z_in)

a_true, z_out_true = layer.predict(z_in_gpu, output_only=False)

# Test the result
(z_in.reshape(len(z_in), -1) == z_out_true.get()).all()

True

# Test Backprop

In [23]:
g_prime = np.random.rand(n_datapoints, *input_shape).astype(np.float32)
new_delta = np.random.rand(n_datapoints, output_nodes).astype(np.float32)

g_prime_gpu = cl_array.to_device(queue, g_prime)
new_delta_gpu = cl_array.to_device(queue, new_delta)

delta_true = layer.get_delta_backprop_(g_prime_gpu, new_delta_gpu)

# Test the result
(delta_true.reshape(len(delta_true), -1).get() == new_delta).all()

True

# Test Weight Gradients

In [24]:
delta = np.random.rand(n_datapoints, *input_shape).astype(np.float32)
prev_z = np.random.rand(n_datapoints, output_nodes).astype(np.float32)

delta_gpu = cl_array.to_device(queue, delta)
prev_z_gpu = cl_array.to_device(queue, prev_z)

# Test if it is running
delta_grad_true, w_grad_true = layer.get_weight_grad_(delta_gpu, prev_z_gpu)